In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import AwsIamRole

workspace = WorkspaceClient()

In [0]:
workspace.storage_credentials.create(
    name='sc_bookstorerawdata'
    ,aws_iam_role=AwsIamRole(role_arn = 'arn:aws:iam::302263052839:role/databricks-s3-ingest-3fcda-db_s3_iam')
)
workspace.storage_credentials.create(
    name='sc_bookstoredatabricks'
    ,aws_iam_role=AwsIamRole(role_arn = 'arn:aws:iam::302263052839:role/databricks-s3-ingest-f00cb-db_s3_iam')
)

workspace.external_locations.create(
    name='el_bookstorerawdata'
    ,url='s3://bookstorerawdata/'
    ,credential_name='sc_bookstorerawdata'
)
workspace.external_locations.create(
    name='el_bookstoredatabricks'
    ,url='s3://bookstoredatabricks/'
    ,credential_name='sc_bookstoredatabricks'
)

spark.sql(f"CREATE CATALOG IF NOT EXISTS bookstore")
spark.sql(f"CREATE DATABASE IF NOT EXISTS bookstore.bronze")
spark.sql(f"CREATE DATABASE IF NOT EXISTS bookstore.silver")
spark.sql(f"CREATE DATABASE IF NOT EXISTS bookstore.gold")
spark.sql(f"CREATE TABLE IF NOT EXISTS bookstore.silver.books (book_id STRING, title STRING, author STRING, price DOUBLE, isActive BOOLEAN, start_date TIMESTAMP, end_date TIMESTAMP)")
spark.sql(f"CREATE TABLE IF NOT EXISTS bookstore.silver.customers (customer_id STRING, email STRING, first_name STRING, last_name STRING, gender STRING, city STRING, country STRING, last_updated TIMESTAMP)")
spark.sql(f"CREATE TABLE IF NOT EXISTS bookstore.silver.orders (order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, email STRING, first_name STRING, last_name STRING, gender STRING, street STRING, city STRING, country STRING, row_time TIMESTAMP, processed_timestamp TIMESTAMP, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>)")


In [0]:
spark.sql(f"DROP TABLE IF EXISTS bookstore.silver.books")
spark.sql(f"DROP TABLE IF EXISTS bookstore.silver.customers")
spark.sql(f"DROP TABLE IF EXISTS bookstore.silver.orders")
spark.sql(f"DROP DATABASE IF EXISTS bookstore.bronze CASCADE")
spark.sql(f"DROP DATABASE IF EXISTS bookstore.silver CASCADE")
spark.sql(f"DROP DATABASE IF EXISTS bookstore.gold CASCADE")
spark.sql(f"DROP CATALOG IF EXISTS bookstore CASCADE")
workspace.external_locations.delete('el_bookstorerawdata')
workspace.external_locations.delete('el_bookstoredatabricks')
workspace.storage_credentials.delete('sc_bookstorerawdata')
workspace.storage_credentials.delete('sc_bookstoredatabricks')